question 1: 

Task: use 17 industry portfolio as regressor, estimate HML and MOM 

1. create HML and MOM using 6 portfolios
2. the returns of HML is factor return of value portoflio; the returns of MOM is factor return of momentum portfolio

*portfolio of each year seems to be constructed based on data at the end of June of the same year, so do the same for this?

For each of HML and MOM: 
1. regress 17 industry portfolio on HML/MOM

2. run one of each type of regression: linear, ridge, and LASSO with all regressor. Pick best 2 out 3 using R2, AIC, and BIC 

3. best 2 x (forward + hybrid) + random forest 


Compare selection criteria across models and across (1a), (1b), and (1c)
1. For selection of optimal model with the same number of predictors we only use SSE, since AIC and BIC penalize more predictors, but for same # variable model they are same as SSE up to some scalar

  

select three best models for HML and for MOM separately for question 1 

  

question 2: 

Task: use 17 industry portfolio as regressor, Comment on the ease or difficulty of replicating the components as opposed to the entire return 

1. use three best model for HML and compare that with accuracy of SV+BV and SG+BG 

2. use three best model for MOM and compare that with accuracy of SH+BH and SL+BL 

  

question 3: 

Task: combine the three procedures, and test accuracy on HML and MOM vs each of the component 

if still have word count, can test accuracy on long-side and short side and/or do pairs combinations of three procedures 


In [77]:
import numpy as np
import pandas as pd
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [78]:
mom = pd.read_csv("factor_return_mom.csv")

hml = pd.read_csv("factor_return_value.csv")

#The 6 Portfolios Formed on Size and Momentum (contains SH BH SL BL)
size_momentum = pd.read_csv("6_Portfolios_size_mom.csv")
size_momentum = size_momentum.iloc[:1174]
size_momentum['Unnamed: 0'] = size_momentum['Unnamed: 0'].astype(int)

#The 6 Portfolios Formed on Size and Value (contains SV BV SG BG)
size_value = pd.read_csv("6_Portfolios_size_value.csv")
#small lo seems to be small growth

#The 17 Industry Portfolios (regressors)
seventeen = pd.read_csv("17_Industry_Portfolios_cleaned.csv")

# Display the first few rows of the DataFrame
print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

   Unnamed: 0   HML
0      192607 -2.43
1      192608  3.82
2      192609  0.13
3      192610  0.70
4      192611 -0.51
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
   Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
0      192607      1.0874   0.9081     -0.0695    5.7168   1.8971    2.0066
1      192608

In [79]:
#make everything start at 192701
hml = hml.iloc[6:]
size_value = size_value.iloc[6:]
seventeen = seventeen.iloc[6:]

print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

# check if times are the same, or some dataset is missing certain dates

    Unnamed: 0   HML
6       192701  4.54
7       192702  2.94
8       192703 -2.61
9       192704  0.81
10      192705  4.73
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
    Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
6       192701     -0.7991  -0.3782      3.9316   -0.2621   0.0321    4.0885
7    

In [80]:
size_momentum['Unnamed: 0'] = pd.to_datetime(size_momentum['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
size_value['Unnamed: 0'] = pd.to_datetime(size_value['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
hml['Unnamed: 0'] = pd.to_datetime(hml['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
mom['Unnamed: 0'] = pd.to_datetime(mom['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
seventeen['Unnamed: 0'] = pd.to_datetime(seventeen['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')

In [81]:
print(seventeen.columns)
seventeen.columns = seventeen.columns.str.strip() #strip spaces
predictors = seventeen.columns[1:]

Index(['Unnamed: 0', 'Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems',
       'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils',
       'Rtail', 'Finan', 'Other'],
      dtype='object')


In [82]:
# Set date as the index
seventeen.set_index('Unnamed: 0', inplace=True)
size_momentum.set_index('Unnamed: 0', inplace=True)
size_value.set_index('Unnamed: 0', inplace=True)
hml.set_index('Unnamed: 0', inplace=True)
mom.set_index('Unnamed: 0', inplace=True)

In [83]:
#group data into 5-year intervals
seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
size_value_grouped = size_value.groupby(pd.Grouper(freq='5Y'))
mom_grouped = mom.groupby(pd.Grouper(freq='5Y'))
hml_grouped = hml.groupby(pd.Grouper(freq='5Y'))

/tmp/ipykernel_1925/903688633.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1925/903688633.py:2: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1925/903688633.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1925/903688633.py:3: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1925/903688633.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_value_grouped = si

In [84]:
group_labels = list(seventeen_grouped.groups.keys())

In [85]:
print(group_labels)

[Period('1927', '5Y-DEC'), Period('1932', '5Y-DEC'), Period('1937', '5Y-DEC'), Period('1942', '5Y-DEC'), Period('1947', '5Y-DEC'), Period('1952', '5Y-DEC'), Period('1957', '5Y-DEC'), Period('1962', '5Y-DEC'), Period('1967', '5Y-DEC'), Period('1972', '5Y-DEC'), Period('1977', '5Y-DEC'), Period('1982', '5Y-DEC'), Period('1987', '5Y-DEC'), Period('1992', '5Y-DEC'), Period('1997', '5Y-DEC'), Period('2002', '5Y-DEC'), Period('2007', '5Y-DEC'), Period('2012', '5Y-DEC'), Period('2017', '5Y-DEC'), Period('2022', '5Y-DEC')]


# Analysis (1a)

## 5-year data

### Linear regression

In [86]:
# linear (variable selection)

linear = LinearRegression(fit_intercept=False)

### forward stepwise selection (use out of sample criteria)

###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
        test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

/tmp/ipykernel_1925/213933389.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/213933389.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/213933389.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [87]:
hml_to_exclude = hml_grouped.get_group(group_labels[0])
mom_to_exclude = mom_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values
mom_5yr_actual = mom[~mom.index.isin(mom_to_exclude.index)]
mom_5yr_actual = mom_5yr_actual.values

In [88]:
#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

print(sse_hml)
print(mom_hml)

Food     13439.711624
Mines    12306.593863
Oil      12209.792863
Clths    13345.427975
Durbl    12650.848402
Chems    13539.275147
Cnsum    13417.337676
Cnstr    13249.643105
Steel    12181.695726
FabPr    13585.194198
Machn    12908.266458
Cars     13772.389498
Trans    10365.938025
Utils    13899.774902
Rtail    13923.576120
Finan    12470.925829
Other    13150.536557
dtype: float64
Food     775444.170624
Mines    768312.851782
Oil      756073.319825
Clths    740956.413708
Durbl    740955.267022
Chems    759370.494591
Cnsum    775775.019893
Cnstr    777340.260450
Steel    742414.093106
FabPr    784235.735592
Machn    761696.347397
Cars     738632.643431
Trans    688929.757482
Utils    804604.511670
Rtail    793685.789396
Finan    738806.241831
Other    792293.654136
dtype: float64


In [89]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


In [90]:
###### two variable

# first selected variable is Trans for both

#remove from predictors
predictors1 = predictors[predictors != 'Trans']

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors1:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/38491543.py:25: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/38491543.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/38491543.py:25: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1925/38491543.py:25: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/38491543.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/38491543.py:25: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


In [91]:
###### three variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)

predictorsh = predictors1[~predictors1.isin(['Other'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/3066780203.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3066780203.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3066780203.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Other


/tmp/ipykernel_1925/3066780203.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/3066780203.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/3066780203.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [92]:
###### four variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Other'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/1986956254.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/1986956254.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/1986956254.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1925/1986956254.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/1986956254.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/1986956254.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [93]:
###### five variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)
# Optimal four variable model is Trans + Other + 'Finan' + 'Machn' (HML), Trans + Cnstr + 'Other' + 'Finan' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Machn'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan', 'Machn']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/359674172.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/359674172.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/359674172.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: FabPr


/tmp/ipykernel_1925/359674172.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/359674172.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/359674172.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [94]:
###### six variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)
# Optimal four variable model is Trans + Other + 'Finan' + 'Machn' (HML), Trans + Cnstr + 'Other' + 'Finan' (MOM)
# Optimal five variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan', 'Machn', 'Chems']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other', 'Finan', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/1768775090.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/1768775090.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/1768775090.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Utils


/tmp/ipykernel_1925/1768775090.py:47: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [95]:
###### seven variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)
# Optimal four variable model is Trans + Other + 'Finan' + 'Machn' (HML), Trans + Cnstr + 'Other' + 'Finan' (MOM)
# Optimal five variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' (MOM)
# Optimal six variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Utils'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan', 'Machn', 'Chems', 'Rtail']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other', 'Finan', 'FabPr', 'Utils']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/3200806418.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3200806418.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3200806418.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1925/3200806418.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [96]:
###### eight variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)
# Optimal four variable model is Trans + Other + 'Finan' + 'Machn' (HML), Trans + Cnstr + 'Other' + 'Finan' (MOM)
# Optimal five variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' (MOM)
# Optimal six variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' (MOM)
# Optimal seven variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' + 'Cars' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' + 'Durbl' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Durbl'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan', 'Machn', 'Chems', 'Rtail', 'Cars']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other', 'Finan', 'FabPr', 'Utils', 'Durbl']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/3718988868.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3718988868.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3718988868.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1925/3718988868.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [97]:
###### nine variable

# Optimal one variable model is Trans for both
# Optimal two variable model is Trans + Other (HML), Trans + Cnstr (MOM)
# Optimal three variable model is Trans + Other + 'Finan' (HML), Trans + Cnstr + 'Other' (MOM)
# Optimal four variable model is Trans + Other + 'Finan' + 'Machn' (HML), Trans + Cnstr + 'Other' + 'Finan' (MOM)
# Optimal five variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' (MOM)
# Optimal six variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' (MOM)
# Optimal seven variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' + 'Cars' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' + 'Durbl' (MOM)
# Optimal eight variable model is Trans + Other + 'Finan' + 'Machn' + 'Chems' + 'Rtail' + 'Cars' + 'Durbl' (HML), \
# Trans + Cnstr + 'Other' + 'Finan' + 'FabPr' + 'Utils' + 'Durbl' + 'Clths' (MOM)

predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Other', 'Finan', 'Machn', 'Chems', 'Rtail', 'Cars', 'Durbl']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Other', 'Finan', 'FabPr', 'Utils', 'Durbl', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        train = explanatory.get_group(group_labels[i]).values
        test = explanatory.get_group(group_labels[i+1]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1925/3881682506.py:36: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3881682506.py:36: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1925/3881682506.py:36: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1925/3881682506.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1925/3881682506.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [ ]:
### backward stepwise selection

### hybrid stepwise selection

In [ ]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-6, 4, 13)

# Ridge regression with cross-validation to find the best alpha
ridge_cv = RidgeCV(alphas=alpha_values, store_cv_values=True)
ridge_cv.fit(X_train, y_train)

# The best alpha value found through cross-validation
print(f"Best alpha: {ridge_cv.alpha_}")

#calculate CV SSE, AIC, BIC

### lasso

In [ ]:
#random forest